In [1]:
# Import libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import warnings

# TF libraries.
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

2023-11-08 18:56:24.387750: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-08 18:56:24.389522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 18:56:24.411782: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 18:56:24.411811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 18:56:24.411825: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
# Get the 2 filepaths.
closed_path = os.path.join('..','data','eyes','close')
open_path = os.path.join('..','data','eyes','open')



In [3]:
img_shape = (260, 400)
print('Images have a shape of (rows {y}, columns {x}) ', img_shape)

Images have a shape of (rows {y}, columns {x})  (260, 400)


In [4]:
# Get number of all .jpg files in each directory.
open_jpg_files = glob.glob(os.path.join(open_path, '*.jpg'))
num_open_jpg_files = len(open_jpg_files)

closed_jpg_files = glob.glob(os.path.join(closed_path, '*.jpg'))
num_closed_jpg_files = len(closed_jpg_files)

In [5]:
X1 = np.zeros((num_open_jpg_files, np.prod(img_shape[:])))

for i in (range(X1.shape[0])):
    file =  os.path.join(open_path,'openedEye' + str(i) +'.jpg')
    # print(file)
    # Import image.
    img = load_img(file, grayscale=True)

    # Turn to array and remove dimensions where dim = 1.
    img = img_to_array(img)
    img = np.squeeze(img)

    # Reshape 2D into 1D and scale by 255.
    x = img.reshape(np.prod(img.shape)) / 255
   
    # Store into X array where each row is an image and each col a pixel
    X1[i] = x
    
# Create the y array for the open train images
y1 = np.ones((X1.shape[0], ))


In [6]:
# Create the X3 array for the closed images.
X3 = np.zeros((num_closed_jpg_files, np.prod(img_shape[:])))

for i in (range(X3.shape[0])):
    file =  os.path.join(closed_path,'closedEye' + str(i) +'.jpg')
    # print(file)
    # Import image.
    img = load_img(file, grayscale=True)

    # Turn to array and remove dimensions where dim = 1.
    img = img_to_array(img)
    img = np.squeeze(img)

    # Reshape 2D into 1D and scale by 255.
    x = img.reshape(np.prod(img.shape)) / 255
    
    # Store into X array where each row is an image and each col a pixel
    X3[i] = x

    
# Create the y array for the open train images
y3 = np.zeros((X3.shape[0], ))


In [7]:
X = np.concatenate([X1, X3])
y = np.concatenate([y1, y3])

In [8]:
# Get a random subsample of the data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [10]:
# Import model
model = tf.keras.models.load_model('../models/open-closed-classifier.h5')


2023-11-08 18:57:40.165023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-08 18:57:40.165382: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 200)               20800200  
                                                                 
 dense_8 (Dense)             (None, 200)               40200     
                                                                 
 dense_9 (Dense)             (None, 200)               40200     
                                                                 
 dense_10 (Dense)            (None, 200)               40200     
                                                                 
 dense_11 (Dense)            (None, 200)               40200     
                                                                 
 dense_12 (Dense)            (None, 200)               40200     
                                                                 
 dense_13 (Dense)            (None, 1)                

In [12]:
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 8ms/step - loss: 1.1212e-07 - accuracy: 1.0000


[1.1211611905537211e-07, 1.0]

In [13]:
# Evaluate on dataset full of open eyes.
X_new = np.zeros((80, np.prod(img_shape[:])))

for i in (range(X_new.shape[0])):
    file =  os.path.join('.','test_img','openEye' + str(i) +'.jpg')
    # print(file)
    # Import image.
    img = load_img(file, grayscale=True)

    # Turn to array and remove dimensions where dim = 1.
    img = img_to_array(img)
    img = np.squeeze(img)

    # Reshape 2D into 1D and scale by 255.
    x = img.reshape(np.prod(img.shape)) / 255
   
    # Store into X array where each row is an image and each col a pixel
    X_new[i] = x
    
# Create the y array for the open train images
y_new = np.ones((X_new.shape[0], ))

In [14]:
X_new.shape

(80, 104000)

In [15]:
y_new.shape

(80,)

In [16]:
model.evaluate(X_new, y_new)

3/3 [==============================] - 0s 8ms/step - loss: 0.8570 - accuracy: 0.9375


[0.8569580316543579, 0.9375]

It can be observed that the validation set exposes an accuracy of 93.75%, which is not bad per-ce but can definitely be improved. To do this, we would have to improve make the model deeper and possibly more complex (variance issue). However, this accuracy is not bad and, for that reason, we will keep the model unchanged.